# Module 2: Mathematical Foundations for Machine Learning

## Introduction

Welcome to the second module in our series on neural networks and language modeling! This notebook is designed for senior developers who are new to machine learning but have strong programming backgrounds.

In this module, we'll explore the mathematical concepts that underpin machine learning algorithms. While you may have encountered some of these concepts before, we'll focus on their specific applications in ML and how they relate to the code you'll write.

### What You'll Learn

- **Probability Distributions**: From discrete to continuous, and their role in ML
- **Information Theory**: Entropy, cross-entropy, and KL divergence
- **Loss Functions**: How to quantify model performance
- **Optimization Basics**: Gradient descent and its variants

Let's start by setting up our environment.


In [5]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import torch

# Set up plotting
plt.style.use("ggplot")
%matplotlib inline

## 1. Probability Distributions

Probability distributions are fundamental to machine learning. They describe the likelihood of different outcomes and form the basis for many ML algorithms.

### Discrete vs. Continuous Distributions

- **Discrete distributions** deal with countable outcomes (like rolling a die)
- **Continuous distributions** deal with uncountable outcomes (like height or weight)

In language modeling, we often work with discrete distributions over characters or words.


In [6]:
# Let's load our names dataset again
with open("../data/names.txt") as f:
    names = f.read().splitlines()

# Convert to lowercase
names = [name.lower() for name in names]

# Look at first letter distribution (discrete)
first_letters = [name[0] for name in names]
letter_counts = {}

for letter in first_letters:
    if letter in letter_counts:
        letter_counts[letter] += 1
    else:
        letter_counts[letter] = 1

# Convert to probabilities
total = len(first_letters)
letter_probs = {k: v / total for k, v in letter_counts.items()}

# Sort by letter
letter_probs = dict(sorted(letter_probs.items()))

# Plot
plt.figure(figsize=(12, 6))
plt.bar(letter_probs.keys(), letter_probs.values())
plt.title("Probability Distribution of First Letters in Names")
plt.xlabel("Letter")
plt.ylabel("Probability")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './names.txt'

### Probability Mass Function (PMF) vs. Probability Density Function (PDF)

For discrete distributions, we use a **Probability Mass Function (PMF)** which gives the probability of each possible outcome.

For continuous distributions, we use a **Probability Density Function (PDF)** which gives the relative likelihood of different outcomes. The probability of any specific value is 0, but the probability of a value falling within a range is the integral of the PDF over that range.

Let's look at some common distributions:


In [ ]:
# Discrete: Binomial Distribution
n, p = 10, 0.5  # number of trials, probability of success
x = np.arange(0, n + 1)
binomial = stats.binom.pmf(x, n, p)

plt.figure(figsize=(12, 6))
plt.bar(x, binomial)
plt.title(f"Binomial Distribution (n={n}, p={p})")
plt.xlabel("Number of Successes")
plt.ylabel("Probability")
plt.show()

# Continuous: Normal Distribution
mu, sigma = 0, 1  # mean and standard deviation
x = np.linspace(mu - 4 * sigma, mu + 4 * sigma, 1000)
normal = stats.norm.pdf(x, mu, sigma)

plt.figure(figsize=(12, 6))
plt.plot(x, normal)
plt.title(f"Normal Distribution (μ={mu}, σ={sigma})")
plt.xlabel("Value")
plt.ylabel("Probability Density")
plt.fill_between(x, normal, alpha=0.3)
plt.show()

### Connection to Machine Learning

In machine learning, we often:

1. **Model data** using probability distributions
2. **Estimate parameters** of these distributions from data
3. **Make predictions** by sampling from or maximizing these distributions

For example, in our character-level language model, we're modeling the probability distribution of the next character given the previous character(s).


## 2. Information Theory

Information theory provides tools for quantifying information and uncertainty. It's particularly useful in machine learning for:

- Measuring the information content of data
- Quantifying the difference between probability distributions
- Defining loss functions for training models

### Entropy: Measuring Uncertainty

**Entropy** measures the average amount of "surprise" or uncertainty in a probability distribution. It's defined as:

$$H(X) = -\sum_{x \in X} p(x) \log p(x)$$

Where:
- $X$ is a random variable
- $p(x)$ is the probability of outcome $x$
- The logarithm is typically base 2 (bits) or base e (nats)

Let's calculate the entropy of our first letter distribution:


In [ ]:
def entropy(probs) -> float:
    """Calculate the entropy of a probability distribution."""
    return -sum(p * np.log2(p) for p in probs if p > 0)


# Calculate entropy of first letter distribution
first_letter_entropy = entropy(letter_probs.values())
print(f"Entropy of first letter distribution: {first_letter_entropy:.4f} bits")

# Compare with uniform distribution
uniform_probs = [1 / 26] * 26  # 26 letters in English alphabet
uniform_entropy = entropy(uniform_probs)
print(f"Entropy of uniform distribution over 26 letters: {uniform_entropy:.4f} bits")

# Maximum possible entropy for 26 outcomes
max_entropy = np.log2(26)
print(f"Maximum possible entropy for 26 outcomes: {max_entropy:.4f} bits")

# How much structure is in our distribution?
print(f"Information content (structure): {max_entropy - first_letter_entropy:.4f} bits")

### Cross-Entropy: Measuring Prediction Quality

**Cross-entropy** measures how well one probability distribution predicts samples from another. It's defined as:

$$H(P, Q) = -\sum_{x} P(x) \log Q(x)$$

Where:
- $P$ is the true distribution
- $Q$ is our predicted distribution

Cross-entropy is minimized when $P = Q$, and in that case, it equals the entropy of $P$.

Let's calculate the cross-entropy between our first letter distribution and a uniform distribution:


In [ ]:
def cross_entropy(true_probs: dict[str, float], pred_probs: dict[str, float]) -> float:
    """Calculate the cross-entropy between two probability distributions."""
    # Ensure both distributions have the same keys
    all_keys = set(true_probs.keys()) | set(pred_probs.keys())

    result = 0
    for k in all_keys:
        p = true_probs.get(k, 0)
        q = pred_probs.get(k, 1e-10)  # Add small epsilon to avoid log(0)
        if p > 0:
            result -= p * np.log2(q)
    return result


# Create a uniform distribution over the same letters
uniform_dist = {k: 1 / len(letter_probs) for k in letter_probs.keys()}

# Calculate cross-entropy
ce = cross_entropy(letter_probs, uniform_dist)
print(f"Cross-entropy between true and uniform distributions: {ce:.4f} bits")

# Compare with entropy
print(f"Entropy of true distribution: {first_letter_entropy:.4f} bits")
print(f"Difference (KL divergence): {ce - first_letter_entropy:.4f} bits")

### KL Divergence: Measuring Distribution Difference

**Kullback-Leibler (KL) divergence** measures how one probability distribution differs from another. It's defined as:

$$D_{KL}(P || Q) = \sum_{x} P(x) \log \frac{P(x)}{Q(x)} = H(P, Q) - H(P)$$

Where:
- $P$ is the true distribution
- $Q$ is our approximation
- $H(P, Q)$ is the cross-entropy
- $H(P)$ is the entropy of $P$

KL divergence is always non-negative and equals zero only when $P = Q$.


In [ ]:
def kl_divergence(p: dict[str, float], q: dict[str, float]) -> float:
    """Calculate the KL divergence between two probability distributions."""
    return cross_entropy(p, q) - entropy(p.values())


# Calculate KL divergence
kl = kl_divergence(letter_probs, uniform_dist)
print(f"KL divergence between true and uniform distributions: {kl:.4f} bits")

# Visualize the distributions
plt.figure(figsize=(12, 6))
plt.bar(
    letter_probs.keys(), letter_probs.values(), alpha=0.7, label="True Distribution"
)
plt.bar(
    uniform_dist.keys(), uniform_dist.values(), alpha=0.5, label="Uniform Distribution"
)
plt.title("Comparison of Distributions")
plt.xlabel("Letter")
plt.ylabel("Probability")
plt.legend()
plt.show()

### Connection to Machine Learning

In machine learning:

- **Entropy** helps us understand the inherent uncertainty in our data
- **Cross-entropy** is commonly used as a loss function for classification problems
- **KL divergence** is used in variational autoencoders and other generative models

For our character-level language model, we'll use cross-entropy as our loss function to measure how well our model predicts the next character.


## 3. Loss Functions

Loss functions quantify how well our model is performing. They measure the difference between our model's predictions and the true values, and we aim to minimize this difference during training.

### Common Loss Functions

#### Classification Problems

- **Cross-Entropy Loss**: For multi-class classification
- **Binary Cross-Entropy**: For binary classification
- **Focal Loss**: For imbalanced classification

#### Regression Problems

- **Mean Squared Error (MSE)**: Sensitive to outliers
- **Mean Absolute Error (MAE)**: More robust to outliers
- **Huber Loss**: Combines MSE and MAE

Let's implement and visualize some of these loss functions:


In [ ]:
# Define a range of predicted probabilities
y_pred = np.linspace(0.001, 0.999, 1000)


# Binary Cross-Entropy Loss
def binary_cross_entropy(y_true: float, y_pred: float) -> float:
    return -y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred)


# Mean Squared Error
def mse(y_true: float, y_pred: float) -> float:
    return (y_true - y_pred) ** 2


# Mean Absolute Error
def mae(y_true: float, y_pred: float) -> float:
    return np.abs(y_true - y_pred)


# Plot loss functions for y_true = 1
y_true = 1
bce_loss = binary_cross_entropy(y_true, y_pred)
mse_loss = mse(y_true, y_pred)
mae_loss = mae(y_true, y_pred)

plt.figure(figsize=(12, 6))
plt.plot(y_pred, bce_loss, label="Binary Cross-Entropy")
plt.plot(y_pred, mse_loss, label="MSE")
plt.plot(y_pred, mae_loss, label="MAE")
plt.title(f"Loss Functions (y_true = {y_true})")
plt.xlabel("Predicted Probability")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

### Negative Log-Likelihood

For our character-level language model, we'll use the **negative log-likelihood** as our loss function. This is equivalent to the cross-entropy loss when dealing with discrete distributions.

The negative log-likelihood is defined as:

$$\mathcal{L} = -\sum_{i} \log P(y_i | x_i)$$

Where:
- $x_i$ is the input (previous character(s))
- $y_i$ is the true next character
- $P(y_i | x_i)$ is the probability our model assigns to the true next character

Let's see how this works with our bigram model:


In [ ]:
# Create a simple bigram model from our names dataset
def build_bigram_model(names: list[str]) -> dict[str, dict[str, float]]:
    # Add start and end tokens
    names = ["<" + name + ">" for name in names]

    # Count bigrams
    bigram_counts = {}
    for name in names:
        for c1, c2 in zip(name, name[1:], strict=False):
            if c1 not in bigram_counts:
                bigram_counts[c1] = {}
            if c2 not in bigram_counts[c1]:
                bigram_counts[c1][c2] = 0
            bigram_counts[c1][c2] += 1

    # Convert to probabilities
    bigram_probs = {}
    for c1 in bigram_counts:
        total = sum(bigram_counts[c1].values())
        bigram_probs[c1] = {
            c2: count / total for c2, count in bigram_counts[c1].items()
        }

    return bigram_probs


# Build the model
bigram_probs = build_bigram_model(names[:1000])  # Use a subset for speed

# Calculate negative log-likelihood on a test name
test_name = "<emma>"
log_likelihood = 0

for c1, c2 in zip(test_name, test_name[1:], strict=False):
    # Get probability of c2 given c1
    prob = bigram_probs.get(c1, {}).get(c2, 1e-10)  # Small epsilon to avoid log(0)
    log_likelihood += np.log2(prob)
    print(f"P({c2}|{c1}) = {prob:.6f}, log2(prob) = {np.log2(prob):.6f}")

# Negative log-likelihood
nll = -log_likelihood
print(f"\nLog-likelihood: {log_likelihood:.6f}")
print(f"Negative log-likelihood: {nll:.6f}")

# Average NLL per character (perplexity = 2^avg_nll)
avg_nll = nll / (len(test_name) - 1)
perplexity = 2**avg_nll
print(f"Average NLL per character: {avg_nll:.6f}")
print(f"Perplexity: {perplexity:.6f}")

### Perplexity

**Perplexity** is a common metric for evaluating language models. It's defined as:

$$\text{Perplexity} = 2^{-\frac{1}{N} \sum_{i=1}^{N} \log_2 P(y_i | x_i)} = 2^{\text{avg\_nll}}$$

Where:
- $N$ is the number of predictions
- $P(y_i | x_i)$ is the probability assigned to the true next character

Perplexity can be interpreted as the weighted average number of choices the model is uncertain about at each step. Lower perplexity means better predictions.


## 4. Optimization Basics

Once we have a loss function, we need to optimize our model parameters to minimize this loss. The most common approach is **gradient descent** and its variants.

### Gradient Descent

Gradient descent is an iterative optimization algorithm that works by:

1. Computing the gradient (derivative) of the loss with respect to each parameter
2. Updating each parameter in the direction of steepest descent
3. Repeating until convergence

The update rule is:

$$\theta_{t+1} = \theta_t - \alpha \nabla_\theta \mathcal{L}(\theta_t)$$

Where:
- $\theta_t$ is the parameter at iteration $t$
- $\alpha$ is the learning rate
- $\nabla_\theta \mathcal{L}(\theta_t)$ is the gradient of the loss with respect to $\theta$

Let's visualize gradient descent on a simple function:


In [ ]:
# Define a simple 2D function
def f(x: float, y: float) -> float:
    return x**2 + y**2


# Compute the gradient
def grad_f(x: float, y: float) -> np.ndarray:
    return np.array([2 * x, 2 * y])


# Create a grid of points
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

# Perform gradient descent
x0, y0 = 4, 4  # Starting point
alpha = 0.1  # Learning rate
max_iter = 20
path = [(x0, y0)]

for i in range(max_iter):
    grad = grad_f(x0, y0)
    x0 -= alpha * grad[0]
    y0 -= alpha * grad[1]
    path.append((x0, y0))

# Plot the function and the gradient descent path
plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, 50, cmap="viridis")
plt.colorbar(label="f(x, y)")
plt.plot(*zip(*path, strict=False), "ro-", markersize=8)
plt.title("Gradient Descent Optimization")
plt.xlabel("x")
plt.ylabel("y")
plt.grid(True)
plt.show()

# Plot the loss over iterations
iterations = range(len(path))
loss_values = [f(x, y) for x, y in path]

plt.figure(figsize=(10, 6))
plt.plot(iterations, loss_values, "bo-")
plt.title("Loss vs. Iterations")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

### Learning Rate

The **learning rate** $\alpha$ controls how big of a step we take in the direction of the gradient. It's a crucial hyperparameter:

- If $\alpha$ is too small, convergence will be slow
- If $\alpha$ is too large, we might overshoot the minimum or diverge

Let's see the effect of different learning rates:


In [ ]:
# Perform gradient descent with different learning rates
learning_rates = [0.01, 0.1, 0.5, 1.0]
max_iter = 20
paths = {}

for alpha in learning_rates:
    x0, y0 = 4, 4  # Starting point
    path = [(x0, y0)]

    for i in range(max_iter):
        grad = grad_f(x0, y0)
        x0 -= alpha * grad[0]
        y0 -= alpha * grad[1]
        path.append((x0, y0))

    paths[alpha] = path

# Plot the function and the gradient descent paths
plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, 50, cmap="viridis")
plt.colorbar(label="f(x, y)")

for alpha, path in paths.items():
    plt.plot(*zip(*path, strict=False), "o-", markersize=6, label=f"α = {alpha}")

plt.title("Gradient Descent with Different Learning Rates")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.show()

# Plot the loss over iterations for each learning rate
plt.figure(figsize=(10, 6))

for alpha, path in paths.items():
    iterations = range(len(path))
    loss_values = [f(x, y) for x, y in path]
    plt.plot(iterations, loss_values, "o-", label=f"α = {alpha}")

plt.title("Loss vs. Iterations for Different Learning Rates")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

### Stochastic Gradient Descent (SGD)

In practice, computing the gradient over the entire dataset can be computationally expensive. **Stochastic Gradient Descent (SGD)** addresses this by:

1. Computing the gradient on a small batch of data
2. Updating the parameters based on this estimate
3. Repeating with different batches

This introduces noise in the optimization process but can lead to faster convergence and better generalization.


### Variants of Gradient Descent

Several variants of gradient descent have been developed to improve convergence:

- **Momentum**: Adds a fraction of the previous update to the current one
- **RMSprop**: Adapts the learning rate for each parameter based on the history of gradients
- **Adam**: Combines momentum and RMSprop

These optimizers are readily available in PyTorch and other ML frameworks.


## 5. Practical Application: Training a Simple Model

Let's put these concepts together by training a simple model on our names dataset. We'll use PyTorch to implement a character-level language model that predicts the next character given the current one.


In [ ]:
# Prepare the data
with open("../../02 - Makemore/names.txt") as f:
    names = f.read().splitlines()

# Convert to lowercase and add start/end tokens
names = ["<" + name.lower() + ">" for name in names]

# Create vocabulary
chars = sorted(list(set("".join(names))))
stoi = dict(zip(chars, range(len(chars))))
itos = dict(zip(range(len(chars)), chars))

# Create training data
xs = []  # Input characters
ys = []  # Target characters (next character)

for name in names:
    for ch1, ch2 in zip(name, name[1:], strict=False):
        xs.append(stoi[ch1])
        ys.append(stoi[ch2])

# Convert to PyTorch tensors
xs = torch.tensor(xs)
ys = torch.tensor(ys)

print(f"Vocabulary size: {len(chars)}")
print(f"Number of examples: {len(xs)}")

In [ ]:
# Define a simple model
class BigramLanguageModel:
    def __init__(self, vocab_size: int) -> None:
        self.vocab_size = vocab_size
        # Initialize weights randomly
        self.W = torch.randn(vocab_size, vocab_size, requires_grad=True)

    def forward(self, idx: torch.Tensor) -> torch.Tensor:
        # One-hot encode the input
        x_one_hot = torch.zeros(len(idx), self.vocab_size)
        x_one_hot.scatter_(1, idx.unsqueeze(1), 1.0)

        # Forward pass
        logits = x_one_hot @ self.W

        return logits
        
    def __call__(self, idx: torch.Tensor) -> torch.Tensor:
        return self.forward(idx)

    @staticmethod
    def loss(logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        # Cross-entropy loss
        loss = torch.nn.functional.cross_entropy(logits, targets)
        return loss

    def update(self, lr: float = 0.1) -> None:
        # Manual gradient descent
        with torch.no_grad():
            self.W -= lr * self.W.grad
            self.W.grad = None


# Initialize model
model = BigramLanguageModel(len(chars))

# Training loop
n_epochs = 100
batch_size = 32
losses = []

for epoch in range(n_epochs):
    # Random batch
    idx = torch.randint(0, len(xs), (batch_size,))

    # Forward pass
    logits = model(xs[idx])
    loss = model.loss(logits, ys[idx])

    # Backward pass
    loss.backward()

    # Update
    model.update(lr=0.1)

    # Track loss
    losses.append(loss.item())

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Plot loss over time
plt.figure(figsize=(10, 6))
plt.plot(losses)
plt.title("Loss vs. Epoch")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.show()

### Generating Names

Now that we've trained our model, let's use it to generate some new names:


In [ ]:
# Generate names
def generate_name(model: BigramLanguageModel, max_len: int = 20) -> str:
    out = []
    idx = stoi["<"]  # Start token

    while True:
        # Forward pass
        x_one_hot = torch.zeros(1, model.vocab_size)
        x_one_hot[0, idx] = 1.0
        logits = x_one_hot @ model.W

        # Apply softmax to get probabilities
        probs = torch.nn.functional.softmax(logits, dim=1)

        # Sample from the distribution
        idx = torch.multinomial(probs, num_samples=1).item()

        # Add to output
        out.append(itos[idx])

        # Check if we're done
        if itos[idx] == ">" or len(out) > max_len:
            break

    return "".join(out[:-1])  # Remove the end token


# Generate 10 names
for _ in range(10):
    name = generate_name(model)
    print(name)

## Summary

In this module, we've covered the mathematical foundations of machine learning:

1. **Probability Distributions**: The building blocks of statistical models
2. **Information Theory**: Tools for measuring uncertainty and model quality
3. **Loss Functions**: Ways to quantify model performance
4. **Optimization**: Techniques for finding the best model parameters

We've also applied these concepts to a practical problem: training a simple character-level language model to generate names.

### Key Takeaways for Senior Developers

- **Probability Theory**: ML models often represent and manipulate probability distributions
- **Information Theory**: Concepts like entropy and cross-entropy provide principled ways to measure model performance
- **Loss Functions**: Different problems require different loss functions
- **Optimization**: Gradient-based methods are the workhorses of deep learning

In the next module, we'll build on these foundations to explore the transition from manual counting to neural network-based approaches for language modeling.